In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import print_function, division
import torch
torch.manual_seed(123)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import models
from sklearn import preprocessing

from skimage import io, transform

import matplotlib.pyplot as plt # for plotting
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm
from sklearn.model_selection import KFold



import argparse
#parser = argparse.ArgumentParser()

#parser.add_argument('train_path', type = str)
#parser.add_argument('model_path', type = str)

#args = parser.parse_args()


from IPython.display import Image
import cv2



import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

plt.ion()   

ohe = {'Ardhachakrasana': 0,'Garudasana': 1,'Gorakshasana': 2,'Katichakrasana': 3,'Natarajasana': 4,
 'Natavarasana': 5,'Naukasana': 6,'Padahastasana': 7,'ParivrittaTrikonasana': 8,'Pranamasana': 9,
 'Santolanasana': 10,'Still': 11,'Tadasana': 12,'Trikonasana': 13,'TriyakTadasana': 14,'Tuladandasana': 15,'Utkatasana': 16,
 'Virabhadrasana': 17,'Vrikshasana': 18}
dhe ={0: 'Ardhachakrasana', 1: 'Garudasana',2: 'Gorakshasana',3: 'Katichakrasana',
 4: 'Natarajasana',5: 'Natavarasana',6: 'Naukasana',7: 'Padahastasana',8: 'ParivrittaTrikonasana',
 9: 'Pranamasana',10: 'Santolanasana',11: 'Still',12: 'Tadasana',13: 'Trikonasana',14: 'TriyakTadasana',
 15: 'Tuladandasana',16: 'Utkatasana',17: 'Virabhadrasana',18: 'Vrikshasana'}


class ImageDataset(Dataset):
    
    def __init__(self, data_csv, root_dir , train = True , img_transform = None):
        self.data_csv = data_csv
        self.img_transform = img_transform
        self.is_train = train
        self.root_dir = root_dir
        
        data = pd.read_csv(data_csv)
        if self.is_train:
            images = data.iloc[:,:-1].to_numpy().flatten()
            labels = data.iloc[:,-1].to_numpy().flatten()
            for (i,aasan) in enumerate(labels):
                labels[i] = ohe[aasan]
        else:
            images = data.iloc[:,:].to_numpy().flatten()
            labels = None
        
        self.images = images
            
        self.labels = labels
        print("Total Images: {}, Data Shape = {}".format(len(self.images), images.shape))
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        im_path = self.images[idx]
        im_path = os.path.join(self.root_dir,im_path)
        image = io.imread(im_path)
        
        if self.is_train:
            label = self.labels[idx]
        else:
            label = -1
        
        image = self.img_transform(image)
        return (image,label)

# Data Loader Usage

BATCH_SIZE = 16 # Batch Size. Adjust accordingly
NUM_WORKERS = 20 # Number of threads to be used for image loading. Adjust accordingly.
train_transforms1 = transforms.Compose([transforms.ToPILImage(),
                                        transforms.Resize(512),
                                        transforms.RandomHorizontalFlip(p = 0.5),
                                        transforms.RandomRotation(degrees = 40),
                                        transforms.RandomPerspective(p=0.5),
                                        transforms.RandomResizedCrop(size=(512, 512),scale=(0.6, 1.3)),
                                        transforms.ToTensor()])  

# Train DataLoader
train_data = "../input/yogapose1/training.csv" # Path to train csv file 

train_dataset = ImageDataset(data_csv = train_data,root_dir = '../input/yogapose1' ,train=True, img_transform = train_transforms1)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [28000, 1011])
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS)
dataloaders = dict()
dataloaders['train'],dataloaders['val'] = train_loader,val_loader
# Test DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


dataset_sizes = dict()
dataset_sizes['train'],dataset_sizes['val'] = 28000,1011
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), "model.pth") #"model.pth"

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    
    model.load_state_dict(best_model_wts)
    return model

#model_ft = Sequential()
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 19)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                        num_epochs=8)



In [ ]:
from __future__ import print_function, division
import torch
torch.manual_seed(123)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import models
from sklearn import preprocessing

from skimage import io, transform

import matplotlib.pyplot as plt # for plotting
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm
from sklearn.model_selection import KFold



import argparse
#parser = argparse.ArgumentParser()

#parser.add_argument('model_path', type = str)
#parser.add_argument('test_path', type = str)
#parser.add_argument('test_pred_path', type = str)

#args = parser.parse_args()


from IPython.display import Image
import cv2



import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   

ohe = {'Ardhachakrasana': 0,'Garudasana': 1,'Gorakshasana': 2,'Katichakrasana': 3,'Natarajasana': 4,
 'Natavarasana': 5,'Naukasana': 6,'Padahastasana': 7,'ParivrittaTrikonasana': 8,'Pranamasana': 9,
 'Santolanasana': 10,'Still': 11,'Tadasana': 12,'Trikonasana': 13,'TriyakTadasana': 14,'Tuladandasana': 15,'Utkatasana': 16,
 'Virabhadrasana': 17,'Vrikshasana': 18}
dhe ={0: 'Ardhachakrasana', 1: 'Garudasana',2: 'Gorakshasana',3: 'Katichakrasana',
 4: 'Natarajasana',5: 'Natavarasana',6: 'Naukasana',7: 'Padahastasana',8: 'ParivrittaTrikonasana',
 9: 'Pranamasana',10: 'Santolanasana',11: 'Still',12: 'Tadasana',13: 'Trikonasana',14: 'TriyakTadasana',
 15: 'Tuladandasana',16: 'Utkatasana',17: 'Virabhadrasana',18: 'Vrikshasana'}


class ImageDataset(Dataset):
    
    def __init__(self, data_csv, root_dir , train = True , img_transform = None):
        self.data_csv = data_csv
        self.img_transform = img_transform
        self.is_train = train
        self.root_dir = root_dir
        
        data = pd.read_csv(data_csv)
        if self.is_train:
            images = data.iloc[:,:-1].to_numpy().flatten()
            labels = data.iloc[:,-1].to_numpy().flatten()
            for (i,aasan) in enumerate(labels):
                labels[i] = ohe[aasan]
        else:
            images = data.iloc[:,:].to_numpy().flatten()
            labels = None
        
        self.images = images
            
        self.labels = labels
        print("Total Images: {}, Data Shape = {}".format(len(self.images), images.shape))
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        im_path = self.images[idx]
        im_path = os.path.join(self.root_dir,im_path)
        image = io.imread(im_path)
        
        if self.is_train:
            label = self.labels[idx]
        else:
            label = -1
        
        image = self.img_transform(image)
        return (image,label)

# Data Loader Usage

BATCH_SIZE = 16 # Batch Size. Adjust accordingly
NUM_WORKERS = 20 # Number of threads to be used for image loading. Adjust accordingly.

                                      
test_transforms = transforms.Compose([transforms.ToPILImage(),transforms.Resize(512),transforms.ToTensor()])
# Train DataLoader
# Test DataLoader
test_data = '../input/yogapose1/test.csv' # Path to test csv file
test_dataset = ImageDataset(data_csv = test_data,root_dir = "../input/yogapose1",train=False, img_transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 19.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 19)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft.load_state_dict(torch.load("./model.pth"))

pred = torch.tensor(()).to(device)
lab = torch.tensor(()).to(device)
model_ft.eval()
model_ft.to(device)
with torch.no_grad():
    n_correct =0
    n_samples =0
    for batch_idx, (images,labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        pred = torch.cat((pred,predicted))
        lab = torch.cat((lab,labels))
        n_correct += (predicted == labels).sum().item()
        n_samples += labels.size(0)
        print(batch_idx)
    acc = 100.0 * n_correct / n_samples

    #saving predictions
    pred = pred.cpu()

imt = pd.read_csv("../input/yogapose1/test.csv")
p1 = imt["name"].to_numpy().reshape(-1,1)
p2 = pred.numpy().reshape(-1,1)
p3 = np.zeros(p2.shape,dtype = "object")
for i in range(p2.shape[0]):
    p3[i] = dhe[p2[i][0]]
p1 = np.concatenate((p1[:-1,:],p3[:-1,:]),axis = 1)
C = ["name","category"]
a =pd.DataFrame(p1).to_csv('submission.csv' ,header = C, index = None) 